## Goal
Use your data science knowledge to explore the data provided and create Linear Regression models to predict how many bikes will be rented based on historical information.
## Course Learning Outcomes (CLO) Assessed
- CLO #1 Explain common models and processing pipelines in machine learning applications
- CLO #2 Apply machine learning algorithms to design solutions for real problems
- CLO #4 Analyse results and solutions to verify their correctness and impact on decision making
## Assessment Criteria and Rubric
This assessment is about data exploration and reasoning over the modelling outcome.
You will likely face overfitting and will have to iterate between data engineering and modelling until you get satisfactory results.
It is taken for granted that the code must be readable and has comments indicating what are you doing. Poor readability will detract points from grading.


In [148]:
import pandas as pd

In [149]:
df = pd.read_csv('Energy_Consumption_Data.csv')
print(df)

       House_ID Month  Area_sq_ft Occupants Heating_Type  Age_of_Building  \
0          7271   Jan       662.0         5          NaN               27   
1           861   Oct      1253.0         2          NaN               13   
2          5391   Dec         NaN         3          Gas               22   
3          5192   Aug      1535.0         1          NaN               16   
4          5735   Sep      1336.0         5          NaN               13   
...         ...   ...         ...       ...          ...              ...   
99995      5681   Nov       818.0         3          Gas               12   
99996      8130   Dec      1521.0         5          NaN               24   
99997      4647   Aug      1086.0         4     Electric               33   
99998      2738   Mar         NaN         1     Electric               13   
99999      7874   Jun      2062.0         2          NaN               13   

      Insulation_Quality  Daily_Average_Consumption_kWh  Season  \
0       

In [150]:
# df.info()
df.head()

,House_ID,Month,Area_sq_ft,Occupants,Heating_Type,Age_of_Building,Insulation_Quality,Daily_Average_Consumption_kWh,Season,Energy_Efficiency_Rating,Tariff_Type,Bill_Amount,Renewable_Energy_Installed,Temperature_Average,Power_Outages,Monthly_Consumption_kWh
0,7271,Jan,662.0,5,NaN,27,Good,14.335539,Spring,2,Time-of-Use,88.741986,Yes,55.467771,1,501.232780
1,861,Oct,1253.0,2,NaN,13,cverage,25.917839,Spring,3,Variable,92.354940,Yes,36.388473,1,742.352878
2,5391,Dec,NaN,3,Gas,22,Poor,33.127843,Summer,6,Fixed,96.708638,No,94.550243,0,1018.708608
3,5192,Aug,1535.0,1,NaN,16,Good,7.451494,Autumn,6,Variable,95.568663,No,62.862087,2,199.420939
4,5735,Sep,1336.0,5,NaN,13,Excellent,34.649571,Winter,10,Variable,NaN,No,93.559405,3,933.151370


In [151]:
# Useful variables
columnNames = ['House_ID', 'Month', 'Area_sq_ft', 'Occupants', 'Heating_Type',
    'Age_of_Building', 'Insulation_Quality',
    'Daily_Average_Consumption_kWh', 'Season', 'Energy_Efficiency_Rating',
    'Tariff_Type', 'Bill_Amount', 'Renewable_Energy_Installed',
    'Temperature_Average', 'Power_Outages', 'Monthly_Consumption_kWh'
    ]

# Understand what is in the columns
columnTarget = ['Occupants']
values = {}
for column in columnTarget:
    values[column] = df[column].unique()
print(values)

{'Occupants': array(['5', '2', '3', '1', '4', 'five', 'fwve', 'bive', 'fiwe', 'fpve',
       'fave', 'fiva', 'fivg', 'mive', 'ftve', 'wive', 'fivt', 'fiye',
       'sive', 'nive', 'fiae', 'jive', 'live', 'cive', 'fsve', 'tive',
       'fize', 'fivc', 'yive', 'fivs', 'fivd', 'fide', 'flve', 'oive',
       'iive', 'kive', 'qive', 'fihe', 'hive', 'fivn', 'fivl', 'fife',
       'fiee', 'fire', 'fqve', 'fbve', 'fije', 'fiie', 'fibe', 'fioe',
       'rive', 'fixe', 'fdve', 'ffve', 'fivr', 'fgve', 'fivm', 'fxve',
       'fiqe', 'fivx', 'fivq'], dtype=object)}


In [152]:
def printColumnValues(df, column_name):
    # Convert the column values to a list
    values = df[column_name].unique().tolist()

    # Print the list
    print('\n'.join(values))

In [153]:
# Initialize an empty list to store the sizes
sizes = []
def dfSize(df):
    # Get the current size of the DataFrame
    current_size = df.shape
    print(f"Current size: {current_size}")
    # If there are previous sizes stored, print the last one
    if sizes:
        print(f"Previous size: {sizes[-1]}")
    # Store the current size
    sizes.append(current_size)

### Cleaning Data To-Do
After using the above function to see what kind of unique variables are within each of the columns. I have summarized how to approach the `data cleaning` 
- ~~`House_ID` - Drop~~
- ~~`Month` - Drop~~
- ~~`Area_sq_ft` - Check Float~~
- ~~`Occupants` - convert 'five' to 5~~
- `Heating_Type` - Word length is word
- `Age_of_Building` - remove --
- `Insulation_Quality` - Word length is word
- `Daily_Average_Consumption_kWh` - no **negatives**, float, no **Null**
- `Season` - Use this to sort
- `Energy_Efficiency_Rating` - 1-10 not too sure how useful
- `Tariff_Type` - Its fucked Word length is Words
- `Bill_Amount` - no **negatives**, float, no **Null**
- `Renewable_Energy_Installed` - Kind of useless
- `Temperature_Average` - not really that useful
- `Power_Outages` - needs to be accounted for
- `Monthly_Consumption_kWh` - cant figure out if its derived from daily consumption

In [154]:
dfSize(df)

Current size: (100000, 16)


In [155]:
df.dtypes

House_ID                           int64
Month                             object
Area_sq_ft                       float64
Occupants                         object
Heating_Type                      object
Age_of_Building                    int64
Insulation_Quality                object
Daily_Average_Consumption_kWh    float64
Season                            object
Energy_Efficiency_Rating           int64
Tariff_Type                       object
Bill_Amount                      float64
Renewable_Energy_Installed        object
Temperature_Average              float64
Power_Outages                      int64
Monthly_Consumption_kWh          float64
dtype: object

In [156]:
# Drop columns that are not useful
# We do not need to know what house ID is consuming the energy
df.drop('House_ID', axis=1, inplace=True)
df.head() # Check if the column was dropped

,Month,Area_sq_ft,Occupants,Heating_Type,Age_of_Building,Insulation_Quality,Daily_Average_Consumption_kWh,Season,Energy_Efficiency_Rating,Tariff_Type,Bill_Amount,Renewable_Energy_Installed,Temperature_Average,Power_Outages,Monthly_Consumption_kWh
0,Jan,662.0,5,NaN,27,Good,14.335539,Spring,2,Time-of-Use,88.741986,Yes,55.467771,1,501.232780
1,Oct,1253.0,2,NaN,13,cverage,25.917839,Spring,3,Variable,92.354940,Yes,36.388473,1,742.352878
2,Dec,NaN,3,Gas,22,Poor,33.127843,Summer,6,Fixed,96.708638,No,94.550243,0,1018.708608
3,Aug,1535.0,1,NaN,16,Good,7.451494,Autumn,6,Variable,95.568663,No,62.862087,2,199.420939
4,Sep,1336.0,5,NaN,13,Excellent,34.649571,Winter,10,Variable,NaN,No,93.559405,3,933.151370


In [157]:
# We do not need to know the month as season is a better general indicator for prediction
df.drop('Month', axis=1, inplace=True)
df.head() # Check if the column was dropped


,Area_sq_ft,Occupants,Heating_Type,Age_of_Building,Insulation_Quality,Daily_Average_Consumption_kWh,Season,Energy_Efficiency_Rating,Tariff_Type,Bill_Amount,Renewable_Energy_Installed,Temperature_Average,Power_Outages,Monthly_Consumption_kWh
0,662.0,5,NaN,27,Good,14.335539,Spring,2,Time-of-Use,88.741986,Yes,55.467771,1,501.232780
1,1253.0,2,NaN,13,cverage,25.917839,Spring,3,Variable,92.354940,Yes,36.388473,1,742.352878
2,NaN,3,Gas,22,Poor,33.127843,Summer,6,Fixed,96.708638,No,94.550243,0,1018.708608
3,1535.0,1,NaN,16,Good,7.451494,Autumn,6,Variable,95.568663,No,62.862087,2,199.420939
4,1336.0,5,NaN,13,Excellent,34.649571,Winter,10,Variable,NaN,No,93.559405,3,933.151370


After dropping `Month` and `House_ID` the data-frame can start to be cleaned.

In [158]:
dfSize(df)

Current size: (100000, 14)
Previous size: (100000, 16)


In [159]:
df = df.dropna(subset=['Area_sq_ft'])  # Drop rows with NaN in 'Area_sq_ft' column
df = df[df['Area_sq_ft'].apply(lambda x: isinstance(x, float) and x > 0)]  # Keep rows where 'Area_sq_ft' is a float and > 0
dfSize(df)

Current size: (94877, 14)
Previous size: (100000, 14)


In [160]:
dfSize(df)
df = df.dropna(subset=['Occupants'])  # Drop rows with NaN in 'Occupants' column

columnValues(df, 'Occupants')
columnTarget = ['Occupants']
values = {}
for column in columnTarget:
    values[column] = df[column].unique()
print(values)

def convert_to_int_or_five(value):
    try:
        return int(value)
    except ValueError:
        return 5

df['Occupants'] = df['Occupants'].apply(convert_to_int_or_five)


dfSize(df)

columnTarget = ['Occupants']
values = {}
for column in columnTarget:
    values[column] = df[column].unique()
print(values)

Current size: (94877, 14)
Previous size: (94877, 14)
['5', '2', '1', '4', '3', 'five', 'fwve', 'fiwe', 'fpve', 'fave', 'fiva', 'fivg', 'mive', 'ftve', 'wive', 'fivt', 'fiye', 'sive', 'nive', 'fiae', 'jive', 'live', 'fsve', 'tive', 'fize', 'fivc', 'yive', 'fivs', 'fivd', 'fide', 'flve', 'oive', 'iive', 'kive', 'cive', 'qive', 'fihe', 'hive', 'fivn', 'fivl', 'fife', 'fiee', 'fire', 'fqve', 'fbve', 'fije', 'fiie', 'fibe', 'rive', 'fixe', 'fdve', 'ffve', 'fivr', 'fivm', 'fxve', 'fiqe', 'fivx', 'fivq', 'fgve']
{'Occupants': array(['5', '2', '1', '4', '3', 'five', 'fwve', 'fiwe', 'fpve', 'fave',
       'fiva', 'fivg', 'mive', 'ftve', 'wive', 'fivt', 'fiye', 'sive',
       'nive', 'fiae', 'jive', 'live', 'fsve', 'tive', 'fize', 'fivc',
       'yive', 'fivs', 'fivd', 'fide', 'flve', 'oive', 'iive', 'kive',
       'cive', 'qive', 'fihe', 'hive', 'fivn', 'fivl', 'fife', 'fiee',
       'fire', 'fqve', 'fbve', 'fije', 'fiie', 'fibe', 'rive', 'fixe',
       'fdve', 'ffve', 'fivr', 'fivm', 'fxve', '

In [161]:
df['Area_sq_ft'] = pd.to_float()
df.dropna(subset=['Area_sq_ft'], inplace = True)

AttributeError: module 'pandas' has no attribute 'to_float'